In [1]:
import os

try:
    os.remove("metastore_db/db.lck")
    os.remove("metastore_db/dbex.lck")
except:
    pass


from pyspark.sql import SparkSession

def build_spark_session(app_name, memory='4g', executors=4):
    return SparkSession.builder\
                      .appName(app_name)\
                      .config('spark.executor.memory', memory)\
                      .config('spark.executor.instances', executors)\
                      .getOrCreate()

spark_session = build_spark_session(app_name='ok-google')

from pyspark.sql import functions as f



1 - joindre les dataframe sur la clé ID_CLIENT en concervant tous les clients du périmètre.
2 - compter le nombre de ID_CLIENT et vérifier qu'il correspond aux nombre d'ID_CLIENT dans la variable perimètre.
3 - Caster les variables continues en double et sauvergarder alors le df obtenu dans le repertoire data sur le cluster.
4 - Pour les variables catégorielles, créer une nouvelle variable qui prend la modalité de la variable courante si elle existe et "NA" sinon.
5- Verifier la cohérence des variables continue. Par exemple pour une variable comme age mettre à -1 tous les ages <0 ou>120ans

Remplacer les valeurs manquantes par "-1" pour le variables categorielles et par la moyenne de la colone pour les variables continues

### Lire les fichiers de données

In [2]:
perimetre = spark_session.read.csv("/home/ousmane/data_clients/sample_perimetre.csv", header=True)
histo_client_raw = spark_session.read.csv("/home/ousmane/data_clients/sample_histo_client.csv", header=True)
histo_train_raw = spark_session.read.csv("/home/ousmane/data_clients/sample_histo_train.csv", header=True)
histo_lowcost_raw = spark_session.read.csv("/home/ousmane/data_clients/sample_histo_lowcost.csv", header=True)
visites_raw = spark_session.read.csv("/home/ousmane/data_clients/sample_visites.csv", header=True)

In [3]:
perimetre.take(5)

[Row(ID_CLIENT='0023d2b0a410eb572de950217a2e0fc661955b5d'),
 Row(ID_CLIENT='0026decd53a30d9b3298d6f4072ad3936bf2ea71'),
 Row(ID_CLIENT='002f0b8e5d22360080866fe1e0e17dda67ff317c'),
 Row(ID_CLIENT='00352dc1e7e43436fc0f6a5f2b34bf48864f0372'),
 Row(ID_CLIENT='005a10c0d3a94096cf4040a8f1fd88fbedbdb8fc')]

In [15]:
perimetre.toPandas().shape

(1084217, 1)

In [17]:
histo_client_raw.take(3)

[Row(ID_CLIENT='000843db32fbaecfbb047ca0bb04b1f9f4d9425a', anciennete='1550', recence_cmd='36', AGE=None, LBL_STATUT_CLT='Grand', LBL_GEO_AIR='Aéroports de Paris Orly', LBL_GRP_SEGMENT_NL='Spectateur', LBL_SEG_COMPORTEMENTAL='Chasseurs Bons Plans', LBL_GEO_TRAIN='Paris', LBL_SEGMENT_ANTICIPATION='Mixte', FLG_CMD_CARTE_1225='0'),
 Row(ID_CLIENT='001338752ea32d9de129c8f8bdf3e2224cf0bd71', anciennete='1667', recence_cmd='25', AGE='35.0', LBL_STATUT_CLT='Grand', LBL_GEO_AIR='Aéroport de Marseille Provence  (MRS)', LBL_GRP_SEGMENT_NL='Spectateur', LBL_SEG_COMPORTEMENTAL='Comportement Pro', LBL_GEO_TRAIN='Marseille', LBL_SEGMENT_ANTICIPATION='Anticipateur', FLG_CMD_CARTE_1225='0'),
 Row(ID_CLIENT='003fb9dca8de374386d0fa97b570950583111931', anciennete='395', recence_cmd='15', AGE='25.0', LBL_STATUT_CLT='Moyen moins', LBL_GEO_AIR='Aéroport de Lyon - Saint Exupéry', LBL_GRP_SEGMENT_NL='Spectateur', LBL_SEG_COMPORTEMENTAL='Rythmes scolaires', LBL_GEO_TRAIN='Lyon', LBL_SEGMENT_ANTICIPATION='Peu A

In [22]:
histo_client_raw.columns

['ID_CLIENT',
 'anciennete',
 'recence_cmd',
 'AGE',
 'LBL_STATUT_CLT',
 'LBL_GEO_AIR',
 'LBL_GRP_SEGMENT_NL',
 'LBL_SEG_COMPORTEMENTAL',
 'LBL_GEO_TRAIN',
 'LBL_SEGMENT_ANTICIPATION',
 'FLG_CMD_CARTE_1225']

In [5]:
histo_client_raw.select("LBL_STATUT_CLT").distinct().show()

+--------------------+
|      LBL_STATUT_CLT|
+--------------------+
|         Moyen moins|
|Non present dans ...|
|    Nouveau prospect|
|            Prospect|
|          Tres petit|
|                null|
|               Petit|
|             Inactif|
|       Nouveau actif|
|               Grand|
|          Tres grand|
|          Moyen plus|
+--------------------+



In [4]:
histo_client_raw.select("LBL_STATUT_CLT").distinct().show()

+--------------------+
|      LBL_STATUT_CLT|
+--------------------+
|         Moyen moins|
|Non present dans ...|
|    Nouveau prospect|
|            Prospect|
|          Tres petit|
|                null|
|               Petit|
|             Inactif|
|       Nouveau actif|
|               Grand|
|          Tres grand|
|          Moyen plus|
+--------------------+



In [23]:
histo_client_raw.select("LBL_GRP_SEGMENT_NL").distinct().show()

+------------------+
|LBL_GRP_SEGMENT_NL|
+------------------+
|              null|
|            Eteint|
|        Spectateur|
|            Acteur|
|        Non defini|
|           Endormi|
+------------------+



In [24]:
histo_client_raw.select("LBL_SEG_COMPORTEMENTAL").distinct().show()

+----------------------+
|LBL_SEG_COMPORTEMENTAL|
+----------------------+
|                  null|
|      Comportement Pro|
|      Exclusifs Agence|
|  Chasseurs Bons Plans|
|  Anticipateurs Met...|
|     Rythmes scolaires|
|              Nouveaux|
|      Sans contraintes|
|         Mono-commande|
+----------------------+



In [30]:
visites.show(5)

+--------------------+---------------------+-------------------+
|           ID_CLIENT|days_since_last_visit|      tx_conversion|
+--------------------+---------------------+-------------------+
|000843db32fbaecfb...|                  8.0| 0.1111111111111111|
|001338752ea32d9de...|                  3.0|0.13043478260869565|
|003fb9dca8de37438...|                 15.0|                1.0|
|004efa6652e570ef6...|                 17.0|              0.125|
|005dd0b718a8f4598...|                 15.0| 0.3333333333333333|
+--------------------+---------------------+-------------------+
only showing top 5 rows



In [6]:
## ecrire une fonction pour transformer les features quantitatives ("anciennete", "recence_cmd", "AGE", etc..) en float

### identifier les variables continues et transformer leurs modalités en double.

In [3]:

client_cols_to_keep = ["ID_CLIENT", 'LBL_STATUT_CLT','LBL_GEO_AIR',
            'LBL_SEG_COMPORTEMENTAL','LBL_GEO_TRAIN','LBL_GRP_SEGMENT_NL',
            'LBL_SEGMENT_ANTICIPATION','FLG_CMD_CARTE_1225']

def cast_columns_of_df(df, cols_to_cast, col_to_keep, cast_type='double'):
    """cast continuous columns into double since all columns are """
    return df.select(col_to_keep + [(df[feature].cast(cast_type))
                    for feature in cols_to_cast if 'ID_CLIENT' not in feature])

histo_train = cast_columns_of_df(histo_train_raw, histo_train_raw.columns,
                                 ["ID_CLIENT"], cast_type='double')
histo_lowcost = cast_columns_of_df(histo_lowcost_raw, histo_lowcost_raw.columns,
                                 ["ID_CLIENT"], cast_type='double')

visites = cast_columns_of_df(visites_raw, visites_raw.columns,
                             ["ID_CLIENT"], cast_type='double')

histo_client = cast_columns_of_df(histo_client_raw,
                                  ["anciennete", "recence_cmd", "AGE"],
                                  client_cols_to_keep,
                                 cast_type='double')


In [4]:
visites = cast_columns_of_df(visites,
                                  ["days_since_last_visit"],
                                  ['ID_CLIENT','tx_conversion'],
                                 cast_type='int')


In [5]:
visites.dtypes


[('ID_CLIENT', 'string'),
 ('tx_conversion', 'double'),
 ('days_since_last_visit', 'int')]

faire une jointure entre les informations des différentes tables.
NB: on conservera tous les clients de la table perimetre.
    En effet, ce sont les cleints qu'on souhaite scorer

###  3) joindre les differentes sources de données en se basant sur les données du périmètre

In [6]:
df=perimetre\
.join(histo_client, on="ID_CLIENT",how='left_outer')\
.join(histo_lowcost, on='ID_CLIENT',how='left_outer')\
.join(histo_train, on='ID_CLIENT',how='left_outer')\
.join(visites,on='ID_CLIENT',how='left_outer')

In [7]:
df.toPandas().shape

(1084217, 24)

Nous avons le même nombre de ligne dans le DataFrame apres jointure que dans la table perimetre

### combien a t'on de features quatitatives, qualitatives 

In [8]:
# definissons une fonction qui prend en entrée un df et renvoie la liste des variables quantitatives et celle des variable qualitatives


def variable_type(x):
    qualif=[]
    contin=[]
    n_cont=0
    n_quali=0
    for i in x.dtypes:
        if i[1]=="string":
            qualif.append(i[0])
            n_quali+=1
        else:
            contin.append(i[0])
            n_cont+=1
    return (contin,qualif)

In [9]:
types=variable_type(df)
types

(['anciennete',
  'recence_cmd',
  'AGE',
  'flg_cmd_lowcost',
  'flg_track_nl_lowcost',
  'flg_track_nl',
  'nb_od',
  'mean_nb_passagers',
  'mean_duree_voyage',
  'mean_mt_voyage',
  'mean_tarif_loisir',
  'mean_classe_1',
  'mean_pointe',
  'mean_depart_we',
  'tx_conversion',
  'days_since_last_visit'],
 ['ID_CLIENT',
  'LBL_STATUT_CLT',
  'LBL_GEO_AIR',
  'LBL_SEG_COMPORTEMENTAL',
  'LBL_GEO_TRAIN',
  'LBL_GRP_SEGMENT_NL',
  'LBL_SEGMENT_ANTICIPATION',
  'FLG_CMD_CARTE_1225'])

In [10]:
quali_col=types[1]
quanti_col=types[0]

In [11]:
quanti_col.pop(3)

'flg_cmd_lowcost'

In [12]:
quali_col.append('flg_cmd_lowcost')

### les differentes modalites de la feature LBL_STATUT_CLT

In [13]:
df.select(df["LBL_STATUT_CLT"]).distinct().show()

+--------------------+
|      LBL_STATUT_CLT|
+--------------------+
|         Moyen moins|
|Non present dans ...|
|    Nouveau prospect|
|            Prospect|
|          Tres petit|
|                null|
|               Petit|
|             Inactif|
|       Nouveau actif|
|               Grand|
|          Tres grand|
|          Moyen plus|
+--------------------+



In [40]:
df.printSchema()

root
 |-- ID_CLIENT: string (nullable = true)
 |-- LBL_STATUT_CLT: string (nullable = true)
 |-- LBL_GEO_AIR: string (nullable = true)
 |-- LBL_SEG_COMPORTEMENTAL: string (nullable = true)
 |-- LBL_GEO_TRAIN: string (nullable = true)
 |-- LBL_GRP_SEGMENT_NL: string (nullable = true)
 |-- LBL_SEGMENT_ANTICIPATION: string (nullable = true)
 |-- FLG_CMD_CARTE_1225: string (nullable = true)
 |-- anciennete: double (nullable = true)
 |-- recence_cmd: double (nullable = true)
 |-- AGE: double (nullable = true)
 |-- flg_cmd_lowcost: double (nullable = true)
 |-- flg_track_nl_lowcost: double (nullable = true)
 |-- flg_track_nl: double (nullable = true)
 |-- nb_od: double (nullable = true)
 |-- mean_nb_passagers: double (nullable = true)
 |-- mean_duree_voyage: double (nullable = true)
 |-- mean_mt_voyage: double (nullable = true)
 |-- mean_tarif_loisir: double (nullable = true)
 |-- mean_classe_1: double (nullable = true)
 |-- mean_pointe: double (nullable = true)
 |-- mean_depart_we: double (

quelles sont les features avec valeurs manquantes
remplacer les valeurs manquantes par -1 pour toutes les features

###  les features avec valeurs manquantes

In [6]:
from pyspark.sql.functions import col, sum

df.select(*(sum(col(c).isNull().cast('int')).alias(c) for c in df.columns)).toPandas()

,ID_CLIENT,LBL_STATUT_CLT,LBL_GEO_AIR,LBL_SEG_COMPORTEMENTAL,LBL_GEO_TRAIN,LBL_GRP_SEGMENT_NL,LBL_SEGMENT_ANTICIPATION,FLG_CMD_CARTE_1225,anciennete,recence_cmd,...,nb_od,mean_nb_passagers,mean_duree_voyage,mean_mt_voyage,mean_tarif_loisir,mean_classe_1,mean_pointe,mean_depart_we,days_since_last_visit,tx_conversion
0,0,78998,162977,155160,163010,79522,157822,10283,55,1484,...,49927,49927,49927,49927,62370,49927,49927,49927,220485,220485


### Remplaçons les valeurs manquantes

In [14]:
def replace_missing_value(dd,qualif_col,continu_col):
    dict_mean={feat:dd.select(f.mean(dd[feat])).collect()[0][0]
              for feat in continu_col}
    return dd.select([f.when(dd[feature].isNotNull(),dd[feature])\
              .otherwise('-1').alias(feature) for feature in qualif_col]\
             +[f.when(dd[feature].isNotNull(), dd[feature])\
             .otherwise(dict_mean[feature]).alias(feature) for feature in continu_col])

In [15]:
df1=replace_missing_value(df,quali_col,quanti_col)

In [57]:
df1.dtypes

[('ID_CLIENT', 'string'),
 ('LBL_STATUT_CLT', 'string'),
 ('LBL_GEO_AIR', 'string'),
 ('LBL_SEG_COMPORTEMENTAL', 'string'),
 ('LBL_GEO_TRAIN', 'string'),
 ('LBL_GRP_SEGMENT_NL', 'string'),
 ('LBL_SEGMENT_ANTICIPATION', 'string'),
 ('FLG_CMD_CARTE_1225', 'string'),
 ('flg_cmd_lowcost', 'string'),
 ('anciennete', 'double'),
 ('mean_nb_passagers', 'double'),
 ('flg_track_nl_lowcost', 'double'),
 ('days_since_last_visit', 'double'),
 ('flg_track_nl', 'double'),
 ('recence_cmd', 'double'),
 ('mean_depart_we', 'double'),
 ('nb_od', 'double'),
 ('mean_pointe', 'double'),
 ('mean_classe_1', 'double'),
 ('mean_mt_voyage', 'double'),
 ('tx_conversion', 'double'),
 ('mean_tarif_loisir', 'double'),
 ('AGE', 'double'),
 ('mean_duree_voyage', 'double')]

In [12]:
from pyspark.sql.functions import col, sum
df1.select(*(sum(col(c).isNull().cast('int')).alias(c) for c in df.columns)).toPandas()

,ID_CLIENT,LBL_STATUT_CLT,LBL_GEO_AIR,LBL_SEG_COMPORTEMENTAL,LBL_GEO_TRAIN,LBL_GRP_SEGMENT_NL,LBL_SEGMENT_ANTICIPATION,FLG_CMD_CARTE_1225,anciennete,recence_cmd,...,nb_od,mean_nb_passagers,mean_duree_voyage,mean_mt_voyage,mean_tarif_loisir,mean_classe_1,mean_pointe,mean_depart_we,tx_conversion,days_since_last_visit
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
def input_df(df):
    ds = df.select('ID_CLIENT',
    f.when(df.LBL_GEO_TRAIN.isin(['Toulouse', 'Lille', 'Dijon',
                                  'Lyon', 'Marseille', 'Paris',
                                  'Nice', 'Limoges','Rouen','Rennes',
                                  'Montpellier', 'Bordeaux', 'Metz',
                                  'Strasbourg']), df.LBL_GEO_TRAIN)\
               .otherwise('na').alias('geo_train'),
    f.when(df.LBL_GEO_AIR.isin(['Aéroports de Paris Orly',
                                'Aéroport de Bâle-Mulhouse / Bassel',
                                'Aéroport Lille Lesquin', 'Aéroport de Rennes',
                                'Aéroport de Nantes Atlantique',
                                'Aéroport de Marseille Provence  (MRS)', 
                                'Aéroport de Bordeaux Mérignac',
                                'Aéroports de Paris Roissy-Charles-de Gaulle', 
                                "Aéroport de Nice Côte d'Azur",
                                'Aéroport de Strasbourg',
                                'Aéroport de Lyon - Saint Exupéry', 
                                'Aéroport de Toulouse Blagnac']), df.LBL_GEO_AIR)\
               .otherwise('na').alias('geo_air'),
    f.when(df.FLG_CMD_CARTE_1225 == '1', '1')\
                   .otherwise('0').alias('cc_jeunes'),
    f.when(df.LBL_STATUT_CLT.isin(['Tres grand', 'Nouveau actif',
                                   'Moyen moins', ' Prospect', ' Petit',
                                   'Inactif', 'Tres petit',
                                   'Nouveau prospect', 'Moyen plus',
                                   'Grand']), df.LBL_STATUT_CLT)\
                   .otherwise('na').alias('segt_rfm'),
    f.when(df.LBL_SEGMENT_ANTICIPATION.isin(['Peu Anticipateur', 'Tres Anticipateur',
                                             'Anticipateur', 'Mixte', 'Non Anticipateur',
                                             'Non Defini']), df.LBL_SEGMENT_ANTICIPATION)\
                   .otherwise('na').alias('segt_anticipation'),
    f.when(df.LBL_SEG_COMPORTEMENTAL.isin(['Mono-commande',
                                           'Comportement Pro',
                                           'Exclusifs Agence', 
                                           'Anticipateurs Methodiques',
                                           'Chasseurs Bons Plans', 
                                           'Rythmes scolaires', 'Nouveaux',
                                           'Sans contraintes']),
           df.LBL_SEG_COMPORTEMENTAL).otherwise('na').alias('segt_comportemental'), 
    f.when(df.LBL_GRP_SEGMENT_NL.isin(['Endormi', 'Spectateur', 'Acteur',
                                       'Eteint', 'Non defini']),
           df.LBL_GRP_SEGMENT_NL).otherwise('na').alias('segt_nl'),
    f.when(((df.AGE > 0) & (df.AGE < 100)), df.AGE)\
                   .otherwise(-1).alias('age'),
    f.when(df.recence_cmd >= 0, df.recence_cmd)\
                   .otherwise(-1).alias('recence_cmd'),
    f.when(((df.mean_duree_voyage > 0) & (df.mean_duree_voyage < 750)),
           df.mean_duree_voyage).otherwise(-1).alias('mean_duree_voyage'),
    f.when(df.days_since_last_visit >= 0, df.days_since_last_visit)\
                   .otherwise(-1).alias('recence_visite'),
    f.when(df.mean_mt_voyage > 0, df.mean_mt_voyage)\
                   .otherwise(-1).alias('mean_mt_voyage'),
    f.when(df.anciennete >= 0, df.anciennete)\
                   .otherwise(-1).alias('anciennete'),
    f.when(df.nb_od > 0, df.nb_od)\
                   .otherwise(-1).alias('nb_od'),
    f.when(df.mean_nb_passagers > 0, df.mean_nb_passagers)\
                   .otherwise(-1).alias('mean_nb_passagers'),
    f.when(df.mean_tarif_loisir >= 0, df.mean_tarif_loisir)\
                   .otherwise(-1).alias('mean_tarif_loisir'),
    f.when(df.mean_classe_1 >= 0, df.mean_classe_1)\
                   .otherwise(-1).alias('mean_classe_1'),
    f.when(df.mean_pointe >= 0, df.mean_pointe)\
                   .otherwise(-1).alias('mean_pointe'),
    f.when(df.mean_depart_we >= 0, df.mean_depart_we)\
                   .otherwise(-1).alias('mean_depart_we'),
    f.when(df.tx_conversion >= 0, df.tx_conversion)\
                   .otherwise(-1).alias('tx_conversion'),
    f.when(df.flg_cmd_lowcost == 1, '1')\
                   .otherwise('0').alias('flg_cmd_lowcost'),
    f.when(df.flg_track_nl_lowcost == 1, '1')\
                   .otherwise('0').alias('flg_track_nl_lowcost'), 
    f.when(df.flg_track_nl == 1, '1')\
                   .otherwise('0').alias('flg_track_nl'))
    
    return ds
df1 = input_df(df1)

###  les differentes valeurs de notre label : flg_cmd_lowcost

In [62]:
df1.groupby('flg_cmd_lowcost').count().show()

+---------------+------+
|flg_cmd_lowcost| count|
+---------------+------+
|              0|979911|
|              1|104306|
+---------------+------+



##### features engineering et modélisation

In [18]:

def preprocessed_df(df, label="flg_cmd_lowcostIndex"):
    max_values_to_define_str_cols = 10
    id_col = 'ID_CLIENT'
    
    dty = dict(df.dtypes)
    str_cols = [k for k, v in dty.items() if v == 'string']
    str_cols.remove(id_col)
    
    for c in str_cols:
        stringIndexer = StringIndexer(inputCol=c, outputCol=c+"Index")
        model_str = stringIndexer.fit(df)
        df = model_str.transform(df).drop(c)

    input_cols = df.columns
    input_cols.remove(id_col)
    input_cols.remove(label)
    
    assembler = VectorAssembler(inputCols=input_cols,
                            outputCol="features")
    df = assembler.transform(df)
    
    featureIndexer = VectorIndexer(inputCol="features", 
                   outputCol="indexedFeatures", 
                   maxCategories=max_values_to_define_str_cols).fit(df)
    return featureIndexer.transform(df), df


data, dff = preprocessed_df(df1)



###  prelevons un sample de data pour notre modelisation

In [19]:
DFS=data.sample(False,0.01,seed=0)

In [20]:
DFS.toPandas().shape

(10954, 26)

In [17]:
from pyspark.ml.feature import StringIndexer, VectorAssembler, VectorIndexer
from pyspark.ml.classification import RandomForestClassifier, LogisticRegression


In [21]:

lr = LogisticRegression(labelCol="flg_cmd_lowcostIndex", 
                        featuresCol="indexedFeatures",elasticNetParam=0.5)


In [35]:
classifier = RandomForestClassifier(labelCol="flg_cmd_lowcostIndex", 
                                    featuresCol="indexedFeatures",
                                    maxDepth=15, numTrees=100)


In [28]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [44]:
def compute_model(DF,clf,label):
    
    (trainingData, testData) = DF.randomSplit([0.7, 0.3])
    
    fitting=clf.fit(trainingData)
    prevision=fitting.transform(testData)
    comparaison=prevision.select(["prediction",label]).rdd
    metric=MulticlassMetrics(comparaison)
    mat_conf=metric.confusionMatrix().toArray()
    
    accuracy=(mat_conf[0][0]+mat_conf[1][1])/mat_conf.sum()
    rappel=mat_conf[0][0]/(mat_conf[0][0]+mat_conf[0][1])
    precision=mat_conf[0][0]/(mat_conf[0][0]+mat_conf[1][0])
    evaluator = BinaryClassificationEvaluator(labelCol="flg_cmd_lowcostIndex",metricName="areaUnderROC")
    AUC=evaluator.evaluate(prevision)
    
    metrics = {"precision":precision,"rappel":rappel,"accuracy":accuracy,"AUC":AUC}
    return (metrics,fitting,prevision,mat_conf)

### Logistic regression

quelle est le label renseigne pour la modelisation?

le Label renseigné est : "flg_cmd_lowcostIndex"

In [53]:
logis = compute_model(DFS,lr,"flg_cmd_lowcostIndex")

#### coefficients du modele

In [58]:
logis[1].coefficients

DenseVector([-0.0024, -0.0085, 0.0012, 0.0154, -0.0178, 0.0001, 0.1598, 0.3202, -0.513, -1.6196, -0.3941, 0.1074, -0.0961, -0.0735, 0.0796, -0.9185, 0.0177, -0.0783, -0.0657, -0.0329, 11.3727, 17.9912])

#### Evaluation du modele

In [61]:
logis[0]

{'precision': 0.96384,
 'rappel': 0.9990049751243781,
 'accuracy': 0.9650812763395545,
 'AUC': 0.8858530366625076}

### Random forest

In [45]:
rand_for=compute_model(DFS,classifier,"flg_cmd_lowcostIndex")

Evaluer les performance de notre modele

In [62]:
rand_for[0]

{'precision': 0.9655515071215635,
 'rappel': 1.0,
 'accuracy': 0.9675202998126171,
 'AUC': 0.9291852188308687}

predire alors les clients lowcoast sur un sample de data n'ayant pas servi à l'apprentissage

In [63]:
rand_for[2].groupBy(["flg_cmd_lowcostIndex"]).count().show()

+--------------------+-----+
|flg_cmd_lowcostIndex|count|
+--------------------+-----+
|                 0.0| 2915|
|                 1.0|  287|
+--------------------+-----+

